# FutureHouse platform client usage example

In [ ]:
import sys
from pathlib import Path

sys.path.insert(0, str(Path("..").resolve()))

In [ ]:
import time

from futurehouse_client import Client, JobNames
from futurehouse_client.models import (
    AuthType,
    JobRequest,
    RuntimeConfig,
    Stage,
)
from ldp.agent import AgentConfig

## Client instantiation

Here we use `auth_type=AuthType.API_KEY` to authenticate with the platform.
Please log in to the platform and go to your user settings to get your API key.

In [ ]:
client = Client(
    stage=Stage.PROD,
    auth_type=AuthType.API_KEY,
    api_key="your-api-key",
)

## Submit a job

Submitting jobs is done by calling the `create_job` method, which receives a `JobRequest` object.

In [ ]:
job_data = JobRequest(
    name=JobNames.from_string("crow"),
    query="What is the molecule known to have the greatest solubility in water?",
)
client.create_job(job_data)

while client.get_job().status != "success":
    time.sleep(5)
print(f"Job status: {client.get_job().status}")
print(f"Job answer: \n{client.get_job().formatted_answer}")

Job status: success
Job answer: 
Question: What is the molecule known to have the greatest solubility in water?

The phosphate prodrug of MSX‑2 is identified as having the greatest water solubility among the molecules with explicit solubility values; it reached 9.0 mg/mL in water—exceeding the 7.3 mg/mL value reported for its L‑valine prodrug counterpart and the lower solubility values for other compounds discussed (jornada2015theprodrugapproach pages 3–5). None of the other sources in the provided context specify an absolute water solubility that surpasses this value.



You can also pass a `runtime_config` to the job, which will be used to configure the agent on runtime.
Here, we will define a agent configuration and pass it to the job. This agent is used to decide the next action to take.
We will also use the `max_steps` parameter to limit the number of steps the agent will take.

In [ ]:
agent = AgentConfig(
    agent_type="SimpleAgent",
    agent_kwargs={
        "model": "gpt-4o",
        "temperature": 0.0,
    },
)
job_data = JobRequest(
    name=JobNames.CROW,
    query="How many moons does earth have?",
    runtime_config=RuntimeConfig(agent=agent, max_steps=10),
)
client.create_job(job_data)

while client.get_job().status != "success":
    time.sleep(5)
print(f"Job status: {client.get_job().status}")
print(f"Job answer: \n{client.get_job().formatted_answer}")

Job status: success
Job answer: 
Question: How many moons does earth have?

Earth has one permanent natural moon—the Moon. All of the cited sources consistently support that the only stable, long‐term satellite of Earth is the Moon, which formed via a giant impact and has been the sole permanent companion since ancient times (jedicke2018earthsminimoonsopportunities pages 1-4; belbruno2005wheredidthe pages 1-4; canup2012formingamoon pages 8-8).

In addition to this permanent satellite, studies have identified that Earth’s gravity sometimes captures small near‐Earth objects on a temporary basis. These temporary satellites, called “minimoons,” can be either temporarily captured orbiters (TCOs) or temporarily captured flybys (TCFs). One notable example is 2006 RH120, which was captured for almost a year before escaping back to a heliocentric orbit (jedicke2018earthsminimoonsopportunities pages 4-6; jedicke2018earthsminimoonsopportunities pages 21-23). Although minimoons represent additiona

# Continue a job

The platform allows to keep asking follow-up questions to the previous job.
To accomplish that, we can use the `runtime_config` to pass the `job_id` of the previous job.

Notice that `create_job` accepts both a `JobRequest` object and a dictionary with keywords arguments.

In [ ]:
job_data = JobRequest(name=JobNames.CROW, query="How many species of birds are there?")

job_id = client.create_job(job_data)
while client.get_job().status != "success":
    time.sleep(5)
print(f"First job status: {client.get_job().status}")
print(f"First job answer: \n{client.get_job().formatted_answer}")

First job status: success
First job answer: 
Question: How many species of birds are there?

The answer depends on which species concept is used. Under the traditional biological species concept (BSC)—which defines species based on reproductive isolation—lists such as the American Ornithologists' Union (AOU) checklist recognize roughly 9,000 to 10,000 species (e.g., 9,159 species in earlier checklists and 10,021 in updated lists) (barrowclough2016howmanykinds pages 2-4, 8-11). In contrast, when using the phylogenetic species concept (PSC)—which defines species as diagnosably distinct evolutionary lineages based on morphological or genetic differences—the same analyses suggest that many currently recognized “species” harbor multiple distinct lineages. Barrowclough et al. (2016) demonstrated that a morphological analysis of an exemplar sample produced an average ratio close to 2:1, yielding an estimated total of roughly 18,000 phylogenetic species worldwide (barrowclough2016howmanykinds 

In [ ]:
continued_job_data = {
    "name": JobNames.CROW,
    "query": "From the previous answer, specifically,how many species of crows are there?",
    "runtime_config": {"continued_job_id": job_id},
}

continued_job_id = client.create_job(continued_job_data)
while client.get_job().status != "success":
    time.sleep(5)
print(f"Continued job status: {client.get_job().status}")
print(f"Continued job answer: \n{client.get_job().formatted_answer}")

Continued job status: success
Continued job answer: 
Question: You are following up on a prior task: How many species of birds are there?

To perform a new task using prior context: From the previous answer, specifically,how many species of crows are there?

Based on the provided context, the best-supported estimate is that there are roughly 40 species in the genus Corvus—the group commonly referred to as crows. This figure is derived from analyses that indicate crows make up around one‐third of the species diversity within the Corvidae family, which comprises about 117 species in total (krol2020crowsandravens pages 1-3). Consequently, if approximately one-third of 117 corvid species are crows, that yields roughly 40 species.

Supporting this figure, regional assessments such as those by DeGrazio (1978) list several crow species in different areas—for example, the Common Raven (Corvus corax) and American Crow (Corvus brachyrhynchos) in North America, along with Carrion Crow (Corvus cor